In [9]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

#GPUID = 1
TRAIN = "/home/jungmo/Documents/autonomous/caffe/bird_image/train/train.txt"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_200.txt"
BIRD_ROOT = "/home/jungmo/Documents/autonomous/caffe/"

BINARYPROTO_PATH = os.path.join(BIRD_ROOT, "db/mean.binaryproto")
NPY_PATH = os.path.join(BIRD_ROOT, "db/mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

problem = open('egg_classified_as_bird.txt', "w")
def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "color8/googlenet_iter_100000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [10]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_200.txt"
evaluate_model("test_8_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_200.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 97.52 
Confusion Matrix:
[[2438    2   17   43]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.92 
Confusion Matrix:
[[4896   10   27   67]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.73 
Confusion Matrix:
[[6589   15   47   87]
 [  42  666   47    7]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.25 
Confusion Matrix:
[[8363   16   70  103]
 [  74 1262  104    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.54 
Confusion Matrix:
[[10766    19    98   125]
 [   74  1262   104     8]
 [    0     0    39     5]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 94.78 
Confusion Matrix:
[[10766    19    98   125]
 [   74  1262   104     8]
 [   1

In [11]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_150.txt"
evaluate_model("test_8_150.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_150.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.20 
Confusion Matrix:
[[2455    2   17   26]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.36 
Confusion Matrix:
[[4918    6   27   49]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.00 
Confusion Matrix:
[[6631    8   37   62]
 [  53  644   60    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.45 
Confusion Matrix:
[[8432    8   43   69]
 [ 115 1213  113    7]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.88 
Confusion Matrix:
[[10857    10    60    81]
 [  115  1213   113     7]
 [    0     0    40     4]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 95.45 
Confusion Matrix:
[[10857    10    60    81]
 [  115  1213   113     7]
 [   29     2  2247  

In [12]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_140.txt"
evaluate_model("test_8_140.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_140.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.24 
Confusion Matrix:
[[2456    3   15   26]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.38 
Confusion Matrix:
[[4919    6   29   46]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.68 
Confusion Matrix:
[[6626    9   42   61]
 [  62  625   69    6]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.09 
Confusion Matrix:
[[8413   11   54   74]
 [ 118 1196  126    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.62 
Confusion Matrix:
[[10840    11    73    84]
 [  118  1196   126     8]
 [    0     0    42     2]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 95.39 
Confusion Matrix:
[[10840    11    73    84]
 [  118  1196   126     8]
 [   23     1  2272  

In [13]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_130.txt"
evaluate_model("test_8_130.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_130.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.32 
Confusion Matrix:
[[2458    4   21   17]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.30 
Confusion Matrix:
[[4915    7   38   40]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.49 
Confusion Matrix:
[[6620    9   49   60]
 [  63  617   78    4]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.86 
Confusion Matrix:
[[8402    9   69   72]
 [ 112 1184  144    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.42 
Confusion Matrix:
[[10829    11    90    78]
 [  112  1184   144     8]
 [    0     0    40     4]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 95.57 
Confusion Matrix:
[[10829    11    90    78]
 [  112  1184   144     8]
 [   31     1  2323  

In [14]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_120.txt"
evaluate_model("test_8_120.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_120.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 97.68 
Confusion Matrix:
[[2442    6   28   24]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.86 
Confusion Matrix:
[[4893    7   49   51]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.32 
Confusion Matrix:
[[6603    9   62   64]
 [  60  621   73    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.58 
Confusion Matrix:
[[8380   11   84   77]
 [ 113 1178  148    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.11 
Confusion Matrix:
[[10793    12   112    91]
 [  113  1178   148     9]
 [    0     0    43     1]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 95.38 
Confusion Matrix:
[[10793    12   112    91]
 [  113  1178   148     9]
 [   26     0  2336  

In [15]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_110.txt"
evaluate_model("test_8_110.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_110.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.32 
Confusion Matrix:
[[2458    4   17   21]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.10 
Confusion Matrix:
[[4905    5   45   45]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.25 
Confusion Matrix:
[[6618    8   57   55]
 [  83  601   75    3]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.52 
Confusion Matrix:
[[8393   10   85   64]
 [ 143 1159  140    6]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.10 
Confusion Matrix:
[[10812    10   107    79]
 [  143  1159   140     6]
 [    1     0    41     2]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 94.96 
Confusion Matrix:
[[10812    10   107    79]
 [  143  1159   140     6]
 [   42     2  2273  

In [16]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_100.txt"
evaluate_model("test_8_100.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_8_100.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.56 
Confusion Matrix:
[[2464    2   16   18]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.56 
Confusion Matrix:
[[4928    4   27   41]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.64 
Confusion Matrix:
[[6645    7   35   51]
 [ 101  603   56    2]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.76 
Confusion Matrix:
[[8449    7   43   53]
 [ 209 1127  106    6]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.33 
Confusion Matrix:
[[10882     9    47    70]
 [  209  1127   106     6]
 [    2     0    32    10]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 93.59 
Confusion Matrix:
[[10882     9    47    70]
 [  209  1127   106     6]
 [   75     0  2030  